In [31]:
class Box:
    # 044464441
    # 5       5
    # 9       7
    # 5       5
    # 244484443
    #
    #
    #
    char_dict = {
        'double':["╔","╗","╚","╝","═","║","╦","╣","╩","╠","╬","═","║"],
        'table':["╔","╗","╚","╝","═","║","╤","╢","╧","╟","┼","─","│"]
    }


## TODO Ensure height and widht are at least 2 (smallest possible box)
    def __init__(self, height, width, style='double'):
        self.height = height
        self.width = width
        self.style = style
        self.chars = self.char_dict[self.style]

    def draw(self):
        string = self.top_row()
        string += self.middle_rows()
        string += self.bottom_row()
        print(string)

    def top_row(self):
        chars = self.chars
        return chars[0]+chars[4]*(self.width-2)+chars[1]+'\n'

    def middle_rows(self):
        return self.middle_row()*(self.height-2)

    def middle_row(self):
        chars = self.chars
        return chars[5]+" "*(self.width-2)+chars[5]+'\n'

    def bottom_row(self):
        chars = self.chars
        return chars[2]+chars[4]*(self.width-2)+chars[3]+'\n'


class TextBox(Box):
    def __init__(self, text):
        self.lines = text.splitlines()
        height = len(self.lines) + 4
        width = max(len(line) for line in self.lines) + 4
        super().__init__(height,width)

    def middle_rows(self):
        rows_string = ""
        for line in self.lines:
            rows_string+=self.middle_row(line)
        return rows_string

    def middle_row(self, line):
        chars = self.char_dict[self.style]
        return chars[5]+" "+line+" "*(self.width-3-len(line))+chars[5]+'\n'

class TableBox(Box):
    def __init__(self, table):
        # table = [[str(item) for item in inner_list] for inner_list in table]
        self.table = table

        self.col_widths =[]
        for i in range(len(table[0])):
            self.col_widths.append(max(len(row[i]) for row in table)+2)

        height = len(table)*2 + 1
        width = sum(self.col_widths)+len(table[0])+1
        self.justify = self.center_justify
        super().__init__(height, width, style='table')

    def left_justify(self, text, width):
        margin = " "*(width-len(text)-2)
        return " "+text+" "+margin

    def right_justify(self, text, width):
        margin = " "*(width-len(text)-2)
        return margin+" "+text+" "

    def center_justify(self, text, width):
        right_margin = " "*(((width-len(text)-2)//2)+(width-len(text))%2)
        left_margin = " "*((width-len(text)-2)//2)
        return left_margin+" "+text+" "+right_margin

    def top_row(self):
        top_string = self.chars[0]
        for width in self.col_widths[:-1]:
            top_string += self.chars[4]*width+self.chars[6]
        top_string += self.chars[4]*self.col_widths[-1]+self.chars[1]+'\n'
        return top_string

    def middle_rows(self):
        middle_string = ""
        for row in self.table[:-1]:
            middle_string += self.middle_row(row)
            middle_string += self.divider_row()
        middle_string += self.middle_row(self.table[-1])
        return middle_string

    def middle_row(self, row):
        row_string = self.chars[5]
        for i, col in enumerate(row[:-1]):
            width = self.col_widths[i]
            row_string += self.justify(col, width) + self.chars[12]

        row_string += self.justify(row[-1], self.col_widths[-1]) + self.chars[5]+'\n'
        return row_string


    def divider_row(self):
        divider_string = self.chars[9]
        for width in self.col_widths[:-1]:
            divider_string += self.chars[11]*width+self.chars[10]
        divider_string += (self.chars[11]*self.col_widths[-1]) +self.chars[7]+'\n'
        return divider_string

    def bottom_row(self):
        bottom_string = self.chars[2]
        for width in self.col_widths[:-1]:
            bottom_string += self.chars[4]*width+self.chars[8]
        bottom_string += self.chars[4]*self.col_widths[-1]+self.chars[3]+'\n'
        return bottom_string

from sympy import *
from sympy.logic.boolalg import truth_table
class LogicTable(TableBox):
    def __init__(self, expressions):
        props = list(set.union(*[exp.atoms() for exp in expressions]))

        def truthy(i):
            return 'True' if i == 1 else 'False'
        table = []
        row = []
        for prop in props:
            row.append(str(prop))
        for exp in expressions:
            row.append(str(exp))
        table.append(row)
        truth_tables = [list(truth_table(expression, props)) for expression in expressions]
        for i in range(len(truth_tables[0])):
            row = [truthy(val) for val in truth_tables[0][i][0]]
            for j in range(len(expressions)):
                row.append(str(truth_tables[j][i][1]))
            table.append(row)
        super().__init__(table)

    def draw_window(self):
        pass

p, q, r = symbols('p, q, r')
expressions = [p|q, p&r|q&(p|r)]

LogicTable(expressions).draw()

╔═══════╤═══════╤═══════╤═══════╤═════════════════════════╗
║   r   │   q   │   p   │ p | q │ (p & r) | (q & (p | r)) ║
╟───────┼───────┼───────┼───────┼─────────────────────────╢
║ False │ False │ False │ False │          False          ║
╟───────┼───────┼───────┼───────┼─────────────────────────╢
║ False │ False │ True  │ True  │          False          ║
╟───────┼───────┼───────┼───────┼─────────────────────────╢
║ False │ True  │ False │ True  │          False          ║
╟───────┼───────┼───────┼───────┼─────────────────────────╢
║ False │ True  │ True  │ True  │          True           ║
╟───────┼───────┼───────┼───────┼─────────────────────────╢
║ True  │ False │ False │ False │          False          ║
╟───────┼───────┼───────┼───────┼─────────────────────────╢
║ True  │ False │ True  │ True  │          True           ║
╟───────┼───────┼───────┼───────┼─────────────────────────╢
║ True  │ True  │ False │ True  │          True           ║
╟───────┼───────┼───────┼───────┼───────

In [32]:
text = "Hello\nWorld"
TextBox(text).draw()

table = [["#","Kayla",'Sandra','Total'],["1","5","7","12"],["2","4","7","11"]]
TableBox(table).draw()

╔═══════╗
║ Hello ║
║ World ║
╚═══════╝

╔═══╤═══════╤════════╤═══════╗
║ # │ Kayla │ Sandra │ Total ║
╟───┼───────┼────────┼───────╢
║ 1 │   5   │   7    │  12   ║
╟───┼───────┼────────┼───────╢
║ 2 │   4   │   7    │  11   ║
╚═══╧═══════╧════════╧═══════╝



In [33]:
class Box:
# 0446441
# 5  5  5
# 944X447
# 5  5  5
# 2448443

    style_dict = {
        'double': "╔╗╚╝═║╦╣╩╠╬",
        'default':"┌┐└┘─│┬┤┴├┼",
        'bold':   "┏┓┗┛━┃┳┫┻┣╋",
        'h-dub':  "╒╕╘╛═│╤╡╧╞╪",
        'v-dub':  "╓╖╙╜─║╥╢╨╟╫",
    }
    def __init__(self, contents=" ", style='default', justification = 'left'):
        self.contents = contents.splitlines()
        self.height = len(self.contents)
        self.width = max([len(line) for line in self.contents])+2
        self.style = self.style_dict[style]
        match justification:
            case 'left':
                self.justify = self.left_justify
            case 'right':
                self.justify = self.right_justify
            case 'center':
                self.justify = self.center_justify
        self.draw_buffer = None

    def left_justify(self, text, width):
        margin = " "*(width-len(text)-2)
        return " "+text+" "+margin

    def right_justify(self, text, width):
        margin = " "*(width-len(text)-2)
        return margin+" "+text+" "

    def center_justify(self, text, width):
        left_margin = " "*(((width-len(text)-2)//2)+(width-len(text))%2)
        right_margin = " "*((width-len(text)-2)//2)
        return left_margin+" "+text+" "+right_margin

    def draw(self):
        if self.draw_buffer:
            print(self.draw_buffer)
        else:
            self.generate_draw_buffer()
            print(self.draw_buffer)

    def generate_draw_buffer(self):
        self.draw_buffer = self.top_string() + self.middle_strings() + self.bottom_string()


# 0446441
# 5  5  5
# 944X447
# 5  5  5
# 2448443

    def top_string(self):
        string = self.style[0]+self.style[4]*(self.width)+self.style[1]+'\n'
        return string

    def middle_strings(self):
        string = ""
        for row in self.contents:
            string += self.middle_string(row)
        return string

    def middle_string(self, content):
        string = self.style[5]+self.justify(content, self.width)+self.style[5]+'\n'
        return string

    def bottom_string(self):
        string = self.style[2]+self.style[4]*self.width+self.style[3]+'\n'
        return string


class BoxTable(Box):
    def __init__(self,table):
        self.table = self.prep_table(table)
        self.col_widths = []
        for col in range(self.num_cols):
            self.col_widths.append(max([len(row[col]) for row in self.table])+2)

        super().__init__(justification='center')

    def prep_table(self, table):
        self.num_rows = len(table)
        self.num_cols = max(len(row) for row in table)
        for row in table:
            while len(row)<self.num_cols:
                row.append("")
        return [[str(item) for item in row] for row in table]

    def top_string(self):
        string = self.style[0]
        print(self.col_widths)
        for col in self.col_widths:
            string += self.style[4]*col+self.style[6]

        string = string[:-1] + self.style[1] + '\n'
        return string

table = [['he llo     there',111],[2,'hi',3]]
BoxTable(table).draw()

[18, 5, 3]
┌──────────────────┬─────┬───┐
│   │
└───┘



In [35]:
props = list(set.union(*[exp.atoms() for exp in expressions]))

def truthy(i):
    return 'True' if i == 1 else 'False'
table = []
row = []
for prop in props:
    row.append(str(prop))
for exp in expressions:
    row.append(str(exp))
table.append(row)
truth_tables = [list(truth_table(expression, props)) for expression in expressions]
for i in range(len(truth_tables[0])):
    row = [truthy(val) for val in truth_tables[0][i][0]]
    for j in range(len(expressions)):
        row.append(str(truth_tables[j][i][1]))
    table.append(row)
print(table)

[['r', 'q', 'p', 'p | q', '(p & r) | (q & (p | r))'], ['False', 'False', 'False', 'False', 'False'], ['False', 'False', 'True', 'True', 'False'], ['False', 'True', 'False', 'True', 'False'], ['False', 'True', 'True', 'True', 'True'], ['True', 'False', 'False', 'False', 'False'], ['True', 'False', 'True', 'True', 'True'], ['True', 'True', 'False', 'True', 'True'], ['True', 'True', 'True', 'True', 'True']]


In [36]:
from tkinter import *
from tkinter import ttk
root = Tk()
frm = ttk.Frame(root, padding =10)
frm.grid()
for i, row in enumerate(table):
    for j, data in enumerate(row):
        ttk.Label(frm, text=data).grid(column=j, row=i,padx=5,pady=5)
#ttk.Label(frm, text="Hello World!").grid(column=0, row=0)
#ttk.Button(frm, text='Quit', command=root.destroy).grid(column=1,row=0)
root.mainloop()

In [37]:
class LogicApp:
    def __init__(self):
        self.main = Tk()
        self.frame = ttk.Frame(self.main)
        self.frame.grid()
        self.input_boxes = [""]

    def draw(self):
        for i in range(len(self.input_boxes)):
            self.plus_button = Button(self.main, text="+",command=self.add_input).grid(row=i,column=0)
            self.min_button = Button(self.main,text="-").grid(row=i,column=1)

            self.exp_entry = Entry(self.main).grid(row=i,column=2)
        mainloop()

    def add_input(self):
        self.input_boxes.append("")
        self.main.update()

LogicApp().draw()


In [1]:
import tkinter as tk
from tkinter import ttk
from sympy import *
from sympy.logic.boolalg import truth_table

class LogicApp(tk.Tk):
    def __init__(self):
        super().__init__()  # Initialize the Tk object
        self.title("Logic Expressions")
        # self.geometry("400x300")
        self.frame=ttk.Frame(self,padding=(4,4,4,4))
        self.frame.grid(sticky='nsew')

        # Store entry boxes for logical expressions
        self.input_boxes = []

        # Set up the frame and widgets
        self.create_widgets()

    def create_widgets(self):
        """Initial setup of widgets."""
        # Add first input box by default

        plus_button = ttk.Button(self.frame, text="+", command=self.add_input,width=3).grid(row=0,column=0)
        min_button = ttk.Button(self.frame, text="-",command=self.remove_input,width=3).grid(row=0,column=1)
        self.add_input()
        # Button to show the truth table in a new window
        self.table_button = ttk.Button(self.frame, text="Show Truth Table", command=self.show_truth_table)
        self.table_button.grid(row=0, column=10, columnspan=2)


    def add_input(self):
        """Add a new entry widget for logical expression input."""
        row = len(self.input_boxes)

        exp_entry = ttk.Entry(self.frame, width=20)
        if row == 0:
            exp_entry.focus_set()
        exp_entry.grid(row=row, column=2, padx=5,pady=2)
        exp_entry.bind("<Return>", self.on_return_key)

        # Add the entry widget to the list for later use
        self.input_boxes.append(exp_entry)

    def on_return_key(self, event):
        self.show_truth_table()

    def remove_input(self):
        if len(self.input_boxes) > 1:
            self.input_boxes[-1].grid_forget()
            self.input_boxes.pop(-1)

    def show_truth_table(self):
        """Open a new window to display the truth table."""
        new_window = tk.Toplevel(self)
        new_window.title("Truth Table")
        table = self.create_table()
        for i, row in enumerate(table):
            for j, data in enumerate(row):
                ttk.Label(new_window, text=data).grid(column=j, row=i,padx=5,pady=5)


    def create_table(self):
        expressions = [parse_expr(expr.get()) for expr in self.input_boxes]
        props = list(set.union(*[exp.atoms() for exp in expressions]))

        def truthy(i):
            return 'True' if i == 1 else 'False'
        table = []
        row = []
        for prop in props:
            row.append(str(prop))
        for exp in expressions:
            row.append(str(exp))
        table.append(row)
        truth_tables = [list(truth_table(expression, props)) for expression in expressions]
        for i in range(len(truth_tables[0])):
            row = [truthy(val) for val in truth_tables[0][i][0]]
            for j in range(len(expressions)):
                row.append(str(truth_tables[j][i][1]))
            table.append(row)
        return table

# Run the application
if __name__ == "__main__":
    app = LogicApp()
    app.mainloop()


In [38]:
from sympy import *


In [40]:
parse_expr('p&q').atoms()

{p, q}